## Get Raw Data for Processing

<a href="https://colab.research.google.com/github/arangoml/networkx-adapter/blob/master/examples/batch_graph_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!git clone -b 0.0.0.2.5.3 https://github.com/arangoml/networkx-adapter.git
!rsync -av networkx-adapter/examples/ ./ --exclude=.git
!pip3 install networkx
!pip3 install matplotlib

## Preprocessing ITSM Data
The purpose of this notebook is to prepare the data in a format suitable for machine learning. The dataset consists of a few numerical and many categorical attributes. The numerical attributes are discretized. The embedding for the categorical values is developed similar to developing embeddings for words in NLP (see https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html). Each categorical value is mapped to a unique integer. The encoded data that is presented to the embedding layer is a sequence of integers, with each integer corresponding to a word, This notebook performs this mapping. It also encodes unknown values to a 'UNKNOWN' category.

In [2]:
import pandas as pd
fp = "data/incident_event_log.csv"
df = pd.read_csv(fp)
df['reassigned'] = df['reassigned'] = df['reassignment_count'].apply(lambda x: 0 if x == 0 else 1)

## Discretize the numerical attributes

In [3]:
numeric = ['sys_mod_count', 'reopen_count']
dfn = df[numeric]
dcols = []
for col in numeric:
    dlabel = 'D_' + col
    labels = [dlabel +'_' + str(c) for c in range(5)]
    dcols.append(dlabel)
    dfn[dlabel] = pd.qcut(dfn[col].rank(method='first'),5, labels = labels, duplicates = 'drop')
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
dfn.head()

,sys_mod_count,reopen_count,D_sys_mod_count,D_reopen_count
0,0,0,D_sys_mod_count_0,D_reopen_count_0
1,2,0,D_sys_mod_count_1,D_reopen_count_0
2,3,0,D_sys_mod_count_2,D_reopen_count_0
3,4,0,D_sys_mod_count_2,D_reopen_count_0
4,0,0,D_sys_mod_count_0,D_reopen_count_0


## Isolate the attributes used for the analysis 
1. Remove the timestamp attributes
2. Remove the numeric attributes. The discretized version of these attributes is added subsequently

In [5]:
attributes = df.columns.tolist()
remove = [ 'made_sla', 'opened_at', 'resolved_at','sys_created_at', 'caller_id', 'closed_at',\
          'notify', 'sys_updated_by','sys_created_by', 'number', 'sys_updated_at', 'reassigned' ]
exclude = remove + numeric
keep = list(set(attributes) - set(exclude)) 
keep

['rfc',
 'vendor',
 'opened_by',
 'category',
 'cmdb_ci',
 'assigned_to',
 'urgency',
 'active',
 'subcategory',
 'knowledge',
 'reassignment_count',
 'impact',
 'contact_type',
 'closed_code',
 'priority',
 'incident_state',
 'location',
 'u_priority_confirmation',
 'problem_id',
 'u_symptom',
 'resolved_by',
 'assignment_group',
 'caused_by']

In [6]:
df_cat_vars = df[keep]
df_cat_vars = df_cat_vars.replace(to_replace = '?', value = 'UNKNOWN')
df_cat_vars =  pd.concat([df_cat_vars, dfn[dcols]], axis = 1)
df['made_sla'] = df['made_sla'].map({True: 1, False: 0})

df = df.reset_index()

In [7]:
df['reassigned'].value_counts()

1    71836
0    69876
Name: reassigned, dtype: int64

In [8]:
cols = df_cat_vars.columns.tolist()
vocab_size = 0
for c in cols:
    print("Num unique vals for category " + str(c) + " = " + str(df_cat_vars[c].nunique()))
    vocab_size += df_cat_vars[c].nunique()
print("Vocab size: %s" % vocab_size)

Num unique vals for category rfc = 182
Num unique vals for category vendor = 5
Num unique vals for category opened_by = 208
Num unique vals for category category = 59
Num unique vals for category cmdb_ci = 51
Num unique vals for category assigned_to = 235
Num unique vals for category urgency = 3
Num unique vals for category active = 2
Num unique vals for category subcategory = 255
Num unique vals for category knowledge = 2
Num unique vals for category reassignment_count = 28
Num unique vals for category impact = 3
Num unique vals for category contact_type = 5
Num unique vals for category closed_code = 18
Num unique vals for category priority = 4
Num unique vals for category incident_state = 9
Num unique vals for category location = 225
Num unique vals for category u_priority_confirmation = 2
Num unique vals for category problem_id = 253
Num unique vals for category u_symptom = 526
Num unique vals for category resolved_by = 217
Num unique vals for category assignment_group = 79
Num uniq

## Recode the categorical values to integers

In [9]:
UNKNOWN_VAL = 1
cat_cols = df_cat_vars.columns.tolist()
cat_int_map = {col: dict() for col in cat_cols}
int_index = 2
for c in cat_cols:
    unique_col_values = df_cat_vars[c].unique().tolist()
    col_int_map = cat_int_map[c]
    for uv in unique_col_values:
        if uv == 'UNKNOWN':
            col_int_map[uv] = UNKNOWN_VAL
        else:
            col_int_map[uv] = int_index
            int_index +=1
    df_cat_vars[c] = df_cat_vars[c].map(cat_int_map[c])    

In [10]:
combined_cat_int_map = dict()
for col in cat_int_map.keys():
    for cat_val, int_map in cat_int_map[col].items():
        combined_cat_int_map[cat_val] = int_map
    

## Write preprocessed raw data to disk

In [11]:
fp_cat_int_map = "data/category_to_integer_map.csv"
df_map = pd.DataFrame(combined_cat_int_map, index = [0])
df_map = df_map.T
df_map = df_map.reset_index()
df_map.columns = ["cat_value", "assigned_integer"]
df_map.to_csv(fp_cat_int_map, index = False)

In [12]:
add_to_cat_vars = ['number','sys_updated_at', 'reassigned'] 
df = pd.concat([df[add_to_cat_vars], df_cat_vars], axis = 1)

In [13]:
df['sys_updated_at'] = pd.to_datetime(df['sys_updated_at']) 

In [14]:
df['sys_updated_at'].dtype

dtype('<M8[ns]')

In [15]:
df = df.sort_values(by = ['number', 'sys_updated_at'])

In [16]:
fp = 'data/pp_batch_incident_event_log.csv'
df.to_csv(fp, index = False)

In [17]:
df['reassigned'].value_counts()

1    71836
0    69876
Name: reassigned, dtype: int64

In [18]:
int_index

2373

## Generate data for learning
The data used for learning has the raw data summarized by incident, i.e. , the raw data for each incident is grouped and summarized. A sample of the data used for learning can be viewed.

In [19]:
dfgb = df.groupby(by = ['number'])
df_pp = df.loc[dfgb.sys_updated_at.idxmax()]
df_pp = df_pp.reset_index()
cols = df_pp.columns.tolist()
cols.remove('index')
df_pp = df_pp[cols]
fprp = "data/pp_recoded_incident_event_log.csv"
df_pp.to_csv(fprp, index = False)

In [20]:
df_pp['reassigned'].value_counts()

0    13986
1    10932
Name: reassigned, dtype: int64

In [21]:
df_pp.head()

,number,sys_updated_at,reassigned,rfc,vendor,opened_by,category,cmdb_ci,assigned_to,urgency,active,subcategory,knowledge,reassignment_count,impact,contact_type,closed_code,priority,incident_state,location,u_priority_confirmation,problem_id,u_symptom,resolved_by,assignment_group,caused_by,D_sys_mod_count,D_reopen_count
0,INC0000045,2016-05-03 12:00:00,0,1,1,187,394,1,1,736,740,741,995,997,1025,1028,1033,1050,1056,1063,1287,1,1541,2066,2282,1,2365,2368
1,INC0000047,2016-06-03 10:00:00,1,1,1,188,395,1,502,736,740,742,995,998,1025,1028,1033,1050,1056,1064,1287,1,1542,2067,2284,1,2366,2368
2,INC0000057,2016-06-03 03:00:00,0,1,1,187,396,1,504,736,740,743,995,997,1025,1028,1034,1050,1056,1065,1287,1289,1542,2068,2283,1,2366,2368
3,INC0000060,2016-07-03 13:00:00,0,1,1,189,397,1,505,736,740,744,995,997,1025,1028,1035,1050,1056,1065,1287,1,1543,2069,2285,1,2365,2368
4,INC0000062,2016-05-03 16:00:00,1,1,1,189,398,1,1,736,740,745,995,998,1026,1028,1036,1051,1056,1066,1287,1,1544,2070,2286,1,2366,2368


In [22]:
vocab_size

2385